In [ ]:
# %%

# 필요한 라이브러리 임포트
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL.Image
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import tf_py_environment, batched_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

# Music Environment 임포트
from music_env import SongCatalog, SongHistoryEnv

In [ ]:
# %%

print(f"TensorFlow version: {tf.version.VERSION}")

In [ ]:
# %%

# 하이퍼파라미터 설정
num_iterations = 30000  # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 200000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 5e-4  # @param {type:"number"}
log_interval = 500  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1500  # @param {type:"integer"}

In [ ]:
# %%

# 데이터 로드 및 전처리
print("데이터 로드 중...")
TOP_K = 500
df = pd.read_csv('listening_history.csv', sep=r'\s+', header=None, names=['user', 'song', 'date', 'time'])
df['timestamp'] = df['date'] + ' ' + df['time']

# 상위 곡 필터링
song_counts = df['song'].value_counts()
top_items = song_counts.head(TOP_K).index.tolist()
df = df[df['song'].isin(top_items)]

# 사용자별 시퀀스 생성
df = df.sort_values(['user', 'timestamp'])
user_groups = df.groupby('user')['song'].apply(list).reset_index()
user_groups.columns = ['user_id', 'listening_history']

print(f"총 사용자 수: {len(user_groups)}")
print(f"상위 곡 수: {len(top_items)}")

In [ ]:
# %%

# Music Environment 생성
print("음악 추천 환경 초기화 중...")
song_catalog = SongCatalog(user_groups, top_items)
print("SVD 모델 훈련 완료")

# 환경 생성
train_py_env = SongHistoryEnv(song_catalog, user_groups, max_steps=10)
eval_py_env = SongHistoryEnv(song_catalog, user_groups, max_steps=10)

In [ ]:
# %%

# 환경 테스트
print('Observation Spec:')
print(train_py_env.observation_spec())

In [ ]:
# %%

print('Reward Spec:')
print(train_py_env.time_step_spec().reward)

In [ ]:
# %%

print('Action Spec:')
print(train_py_env.action_spec())

In [ ]:
# %%

# 환경 동작 확인
time_step = train_py_env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)
next_time_step = train_py_env.step(action)
print('Next time step:')
print(next_time_step)

In [ ]:
# %%

# TensorFlow 환경으로 래핑
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [ ]:
# %%

# Q-Network 설정
fc_layer_params = (64, 32)
action_tensor_spec = tensor_spec.from_spec(train_py_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

print(f"액션 수: {num_actions}")

# Dense layer 헬퍼 함수 정의
def dense_layer(num_units):
    return tf.keras.layers.Dense(
        num_units,
        activation=tf.keras.activations.relu,
        kernel_initializer=tf.keras.initializers.VarianceScaling(
            scale=2.0, mode='fan_in', distribution='truncated_normal'))

# Q-Network 구성
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [ ]:
# %%

# DQN 에이전트 생성
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [ ]:
# %%

# 정책 정의
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [ ]:
# %%

# 랜덤 정책 (비교용)
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

In [ ]:
# %%

def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    total_hits = 0
    total_steps = 0
    
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0
        episode_hits = 0
        episode_steps = 0

        while not time_step.is_last():
            # 핵심 수정 1: Shape 문제 해결
            if len(time_step.observation.shape) == 3:
                time_step = time_step._replace(
                    observation=tf.squeeze(time_step.observation, axis=1)
                )
            
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            
            # 핵심 수정 2: Format 오류 방지
            episode_return += float(time_step.reward)
            
            # 적중률 계산을 위한 추가
            if float(time_step.reward) > 0:
                episode_hits += 1
            episode_steps += 1
            
        total_return += episode_return
        total_hits += episode_hits
        total_steps += episode_steps

    avg_return = total_return / num_episodes
    hit_rate = total_hits / total_steps if total_steps > 0 else 0.0
    
    return float(avg_return), float(hit_rate)
    

In [ ]:
# %%

compute_avg_return(eval_env, random_policy, num_eval_episodes)


In [ ]:
# %%

# Replay Buffer 설정
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
    replay_buffer.py_client,
    table_name,
    sequence_length=2)

In [ ]:
# %%

# 데이터 수집 스펙 확인
print("Collect data spec:")
print(agent.collect_data_spec)

In [ ]:
initial_collect_driver = py_driver.PyDriver(
    train_py_env,
    py_tf_eager_policy.PyTFEagerPolicy(
        random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps)

# 초기 경험 데이터 수집
print("초기 경험 데이터 수집 중...")
initial_collect_driver.run(train_py_env.reset())
print(f"초기 {initial_collect_steps}개 스텝 수집 완료")


In [ ]:
# %%

# 데이터셋 설정
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

iterator = iter(dataset)
print("데이터셋 준비 완료")

In [ ]:
# %%

# 훈련 시작
print("DQN 훈련 시작...")

# 최적화를 위한 그래프 함수 래핑
agent.train = common.function(agent.train)

# 훈련 스텝 카운터 초기화
agent.train_step_counter.assign(0)

# 훈련 전 에이전트 정책 평가
print("훈련 전 에이전트 평가:")
avg_return, hit_rate = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
print(f"초기 - 평균 리턴: {avg_return:.3f}, 적중률: {hit_rate:.3f}")

returns = [avg_return]
hit_rates = [hit_rate]  


# 환경 초기화
time_step = train_py_env.reset()

# 데이터 수집 드라이버 생성
collect_driver = py_driver.PyDriver(
    train_py_env,
    py_tf_eager_policy.PyTFEagerPolicy(
        agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

# 훈련 루프 - 데이터 수집 및 에이전트 훈련

# 손실 값 추적을 위한 리스트 추가
losses = []

# 훈련 루프에서 손실 값 저장 (위의 훈련 루프 수정)
for i in range(num_iterations):
    # 한 스텝 데이터 수집 (드라이버 사용)
    time_step, _ = collect_driver.run(time_step)
    
    # 리플레이 버퍼에서 배치 샘플링 및 에이전트 훈련
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss
    
    step = agent.train_step_counter.numpy()
    
    # 손실 값 저장
    if step % log_interval == 0:
        losses.append(float(train_loss))
        print(f'스텝 = {step}: 손실 = {train_loss:.3f}')
    
    # 성능 평가
    if step % eval_interval == 0:
        avg_return, hit_rate = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
        print(f'스텝 = {step}: 평균 리턴 = {avg_return:.3f}, 적중률 = {hit_rate:.3f}')
        returns.append(avg_return)
        hit_rates.append(hit_rate)
    
    
print("훈련 완료!")

In [ ]:
# 더 안정적인 성능 평가

def stable_evaluation(environment, policy, num_runs=5, episodes_per_run=20):
    """여러 번 실행해서 평균과 표준편차 계산"""
    all_returns = []
    all_hit_rates = []
    
    for run in range(num_runs):
        run_returns = []
        run_hits = 0
        run_steps = 0
        
        for episode in range(episodes_per_run):
            time_step = environment.reset()
            episode_return = 0.0
            episode_hits = 0
            episode_steps = 0
            
            while not time_step.is_last():
                if len(time_step.observation.shape) == 3:
                    time_step = time_step._replace(
                        observation=tf.squeeze(time_step.observation, axis=1)
                    )
                
                action_step = policy.action(time_step)
                time_step = environment.step(action_step.action)
                
                episode_return += float(time_step.reward)
                if float(time_step.reward) > 0:
                    episode_hits += 1
                episode_steps += 1
                
            run_returns.append(episode_return)
            run_hits += episode_hits
            run_steps += episode_steps
        
        avg_return = np.mean(run_returns)
        hit_rate = run_hits / run_steps if run_steps > 0 else 0
        
        all_returns.append(avg_return)
        all_hit_rates.append(hit_rate)
        
        print(f"Run {run+1}: 평균 리턴 = {avg_return:.3f}, 적중률 = {hit_rate:.3f}")
    
    final_return = np.mean(all_returns)
    return_std = np.std(all_returns)
    final_hit_rate = np.mean(all_hit_rates)
    hit_rate_std = np.std(all_hit_rates)
    
    return final_return, return_std, final_hit_rate, hit_rate_std

# 안정적인 최종 성능 비교
print("\n=== 안정적인 최종 성능 비교 ===")
print("랜덤 정책 (5회 평균):")
random_return, random_return_std, random_hit, random_hit_std = stable_evaluation(
    eval_env, random_policy, num_runs=5, episodes_per_run=20)
print(f"  평균 리턴: {random_return:.3f} ± {random_return_std:.3f}")
print(f"  적중률: {random_hit:.3f} ± {random_hit_std:.3f}")

print("\n훈련된 DQN 에이전트 (5회 평균):")
agent_return, agent_return_std, agent_hit, agent_hit_std = stable_evaluation(
    eval_env, agent.policy, num_runs=5, episodes_per_run=20)
print(f"  평균 리턴: {agent_return:.3f} ± {agent_return_std:.3f}")
print(f"  적중률: {agent_hit:.3f} ± {agent_hit_std:.3f}")

improvement_return = agent_return - random_return
improvement_hit = agent_hit - random_hit

print(f"\n개선도:")
print(f"  리턴 개선: {improvement_return:.3f} ({improvement_return/random_return*100:.1f}%)")
print(f"  적중률 개선: {improvement_hit:.3f} ({improvement_hit/random_hit*100:.1f}%)")

# 통계적 유의성 검정
from scipy import stats

def statistical_significance_test(policy1, policy2, environment, num_episodes=50):
    """두 정책의 성능 차이가 통계적으로 유의한지 검정"""
    returns1 = []
    returns2 = []
    
    for _ in range(num_episodes):
        # Policy 1 평가
        time_step = environment.reset()
        episode_return = 0.0
        while not time_step.is_last():
            if len(time_step.observation.shape) == 3:
                time_step = time_step._replace(
                    observation=tf.squeeze(time_step.observation, axis=1)
                )
            action_step = policy1.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += float(time_step.reward)
        returns1.append(episode_return)
        
        # Policy 2 평가
        time_step = environment.reset()
        episode_return = 0.0
        while not time_step.is_last():
            if len(time_step.observation.shape) == 3:
                time_step = time_step._replace(
                    observation=tf.squeeze(time_step.observation, axis=1)
                )
            action_step = policy2.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += float(time_step.reward)
        returns2.append(episode_return)
    
    # t-검정 수행
    t_stat, p_value = stats.ttest_rel(returns2, returns1)  # 순서 주의: agent - random
    
    print(f"\n=== 통계적 유의성 검정 ===")
    print(f"t-통계량: {t_stat:.3f}")
    print(f"p-값: {p_value:.6f}")
    
    if p_value < 0.05:
        print("✅ 통계적으로 유의한 개선 (p < 0.05)")
    elif p_value < 0.1:
        print("⚠️ 경계선상의 개선 (p < 0.1)")
    else:
        print("❌ 통계적으로 유의하지 않음 (p >= 0.1)")
    
    return t_stat, p_value

# 통계적 유의성 검정 실행 (선택사항)
t_stat, p_value = statistical_significance_test(random_policy, agent.policy, eval_env)